# Inverse Probability Weighting and Doubly Robust Estimation

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Estimadores Baseados em Imputação
  * Aplicação no Python
* Outcome Regression
  * Aplicação no Python
* Doubly Robust Estimation
  * Aplicação no Python
* Boas Práticas




**Observações:** O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer *feedback* ou sugestão de melhoria.

---





## Propensity Score Weighting

### Definição formal do Propensity Score

O **Propensity Score** é definido como a probabilidade condicional de uma unidade receber o tratamento dado o vetor de covariáveis observadas:

$$
p(X_i) = \Pr(D_i = 1 \mid X_i)
$$

onde:

- $D_i \in \{0, 1\}$ é o indicador de tratamento;
- $X_i$ é o vetor de covariáveis observadas.

A ideia central de **Propensity Score Weighting** é reponderar a amostra de forma a criar uma população *pseudo‐randomizada*, em que a distribuição das covariáveis seja balanceada entre tratados e não tratados.

## Inverse Probability of Treatment Weight (IPTW)

### 1. Pesos para o **ATE** (Average Treatment Effect)

O peso **IPTW** para o ATE é definido como:

$$
w_i^{ATE} = \frac{D_i}{p(X_i)} + \frac{1 - D_i}{1 - p(X_i)}
$$

**Na prática**:
- Tratados ($D_i = 1$) recebem peso $1 / p(X_i)$  
- Controles ($D_i = 0$) recebem peso $1 / (1 - p(X_i))$


### 2. Pesos para o **ATT** (Average Treatment Effect on the Treated)

O peso **IPTW** para o ATT é definido como:

$$
w_i^{ATT} = D_i + (1 - D_i) \cdot \frac{p(X_i)}{1 - p(X_i)}
$$

**Na prática**:
- Tratados ($D_i = 1$) recebem peso = 1  
- Controles ($D_i = 0$) recebem peso $p(X_i) / (1 - p(X_i))$


### Interpretação dos pesos

- **No ATE**:  
  - Unidades tratadas com baixa probabilidade de tratamento recebem peso alto.  
  - Unidades de controle com baixa probabilidade de não tratamento recebem peso alto.  

- **No ATT**:  
  - Tratados sempre têm peso 1.  
  - Controles são reponderados para se assemelhar à distribuição de covariáveis dos tratados.

### Hipóteses de identificação

1. **Ignorabilidade (ou não confusão)**:  
   $$
   (Y_i(1), Y_i(0)) \perp D_i \mid X_i
   $$  
   Isto é, não existem confundidores não observados dados $X_i$.

2. **Sobreposição (ou suporte comum)**:  
   $$
   0 < p(X_i) < 1 \quad \forall i
   $$  
   Cada unidade tem probabilidade positiva de receber ambos os estados de tratamento.

### Estimação do efeito causal ponderado

O **ATE ponderado** pode ser visto como:

> Média ponderada dos resultados observados entre os tratados  
> **menos**  
> Média ponderada dos resultados observados entre os controles.

Matematicamente:

$$
\widehat{\tau}_{ATE} =
\underbrace{\frac{\sum_{i=1}^n w_i^{ATE} D_i Y_i}{\sum_{i=1}^n w_i^{ATE} D_i}}_{\text{Média ponderada nos tratados}}
-
\underbrace{\frac{\sum_{i=1}^n w_i^{ATE} (1-D_i) Y_i}{\sum_{i=1}^n w_i^{ATE} (1-D_i)}}_{\text{Média ponderada nos controles}}
$$

O **ATT** pode ser obtido substituindo $w_i^{ATE}$ por $w_i^{ATT}$ na fórmula acima, interpretando o resultado como efeito médio nos tratados.


In [ ]:
# ATE
df['W_ATE'] = df['Treated'] / df['pscore'] + (1 - df['Treated']) / (1 - df['pscore'])

# ATT
df['W_ATT'] = df['Treated'] * 1 + (1 - df['Treated']) * (df['pscore'] / (1 - df['pscore']))


## Doubly Robust Estimation (DR)

A estimativa duplamente robusta combina uma forma de "Outcome Regression" com um modelo de ponderação (ou seja, utilizando o escore de propensão) para estimar o efeito causal sobre um resultado. Quando usados individualmente para estimar um efeito causal, os métodos de regressão de resultados e escore de propensão são não enviesados apenas se o modelo estatístico for especificado corretamente. O **estimador duplamente robusto** combina estas 2 abordagens de modo que apenas 1 dos 2 modelos precisa ser especificado corretamente para obter um estimador de efeito não-viesado.

A especificação correta do modelo de regressão é um pressuposto fundamental na análise econométrica. Quando o objetivo é ajustar o fator de confusão, o estimador é consistente (e, portanto, assintoticamente não-enviesado) se o modelo refletir as verdadeiras relações entre a exposição e os fatores de confusão com o resultado. Na prática, nunca poderemos saber se algum modelo específico representa com precisão essas relações. **A estimativa duplamente robusta combina regressão de resultados com ponderação pelo escore de propensão (PS), de modo que o estimador seja robusto à especificação incorreta de um (mas não de ambos) desses modelos**.

**Outcome Regression Approach**

Vimos que:

$$ \hat{\beta}_{ATE}^{OR} = E[\mu_{1}(X) - \mu_{0}(X)] + E[(Y_{1} - \mu_{1}(X)) - (Y_{0} - \mu_{0}(X))] $$

e

$$ \hat{\beta}^{OR}_{ATT} = E[\mu_{1}(X^{1}) - \mu_{0}(X^{1})] + E[(Y_{1} - \mu_{1}(X^{1})) - (Y_{0} - \mu_{0}(X^{1}))] $$




**Approach de Inverse Probability Weighting (IPW)**

Nesta abordagem, o viés de confusão é ajustado por meio de técnicas de matching (pareamento) e ponderação pelo escore de propensão (Peso = $W$). As ponderações são calculadas da seguinte forma:

$$ W_{ATE} = \frac{D}{\hat{p}(X)} + \frac{1-D}{1-\hat{p}(X)} $$

e,

$$ W_{ATT} = D + (1-D)\frac{\hat{p}(X)}{1-\hat{p}(X)} $$


**Approach do Doubly Robust Estimation (DR)**

A abordagem Doubly Robust Estimation (DR) combina as vantagens das abordagens de Outcome Regression e de Ponderação pela Probabilidade Inversa. Isso proporciona uma maior robustez aos resultados. Os estimadores duplamente robustos para o Average Treatment Effect (ATE) e o Average Treatment Effect on the Treated (ATT) são dados pelas seguintes fórmulas:

* **Doubly Robust Estimation for Average Treatment Effect (ATE)**

$$ \hat{\beta_{ATE}^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) \right] + \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] $$

* **Doubly Robust Estimation for Average Treatment Effect on the Treated (ATT)**

$$ \hat{\beta_{ATT}^{DR}} = \mathbb{E} \left[ (\mu_1 (X) - \mu_0 (X)) \right] + \mathbb{E} \left[ D(Y_{1} - \mu_1 (X)) - \frac{(1-D)\hat{p}(X)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] $$

Repare que realizamos um "ajuste" nos resíduos da regressão de resultados, ponderando-os pelo escore de propensão. Isso garante que o estimador seja robusto à especificação incorreta de um dos modelos. O "ajuste" é essencialmente um estimador IPW realizado sobre os resíduos.

Por isso que o Doubly Robust Estimation também é conhecido como Augmented Inverse Probability Weighting (AIPW).

**Por que a o estimador Duplamente Robusto (*Augmented Inverse Probability Weighting* - AIPW) é tão atraente?**

A razão é que só precisamos de uma das duas previsões, *$\hat{\mu}$* ou *$\hat{p}$*, para que a estimativa seja correta (não enviesada/imparcial). 
* Se ambos os modelos estiverem corretos, o estimador será mais eficiente do que qualquer um dos modelos sozinho. 
* Se um dos modelos estiver errado, o estimador ainda será consistente, desde que o outro modelo esteja correto. 

Isso é uma grande vantagem em relação a outras abordagens, como a regressão de resultados ou a ponderação pelo escore de propensão, que exigem que ambos os modelos estejam corretos para que o estimador seja consistente.

Suponha que $\hat{\mu}$ esteja especificado corretamente. Então $E[\hat{\mu}^{d}(x)=E[Y|X=x, D=d]$ , então o estimador DR é consistente, mesmo que o modelo de propensão $\hat{p}$ esteja mal especificado.

$$ \hat{\beta^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) +  \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] = $$

$$  =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) \right] = $$

$$  =  \mathbb{E} \left[ Y^{1} - Y^{0} \right] = $$

$$ = \beta $$

A intuição é que, se $\hat{\mu}$ está **especificado corretamente é imparcial e o fator de ajuste desaparece**, uma vez que os resíduos convergem para zero.


Por outro lado, suponha $\hat{p}$ está especificado corretamente, ou seja, $E[\hat{p}(X)]=P(D=1|X)$, então o estimador DR é consistente, mesmo que o modelo de resultados $\hat{\mu}$ esteja mal especificado.

$$ \hat{\beta^{DR}} =  \mathbb{E} \left[ (\mu_1(X) - \mu_0 (X)) +  \frac{D}{\hat{p}(X)}.(Y_{1} - \mu_1 (X)) - \frac{(1-D)}{1-\hat{p}(X)}.(Y_{0} - \mu_0 (X)) \right] = $$

$$ \mathbb{E} \left[ \mu_1(X) - \mu_0 (X) + \frac{D}{\hat{p}(X)}.Y_{1} - \frac{D}{\hat{p}(X)}.\mu_1 (X) - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \frac{(1-D)}{1-\hat{p}(X)}.\mu_0 (X) \right] = $$


$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) - \frac{D}{\hat{p}(X)}.\mu_1 (X) + \frac{(1-D)}{1-\hat{p}(X)}.\mu_0 (X) - \mu_0 (X) \right] = $$


$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) (1 - \frac{D}{\hat{p}(X)}) + \mu_0 (X)(\frac{(1-D)}{1-\hat{p}(X)} - 1) \right] = $$

$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} + \mu_1(X) (\frac{\hat{p}(X) - D}{\hat{p}(X)}) + \mu_0 (X)(\frac{(1-D)- (1-\hat{p}(X))}{1-\hat{p}(X)}) \right] = $$

$$ \mathbb{E} \left[ \frac{D}{\hat{p}(X)}.Y_{1} - \frac{(1-D)}{1-\hat{p}(X)}Y_{0} \right] = $$

$$ \mathbb{E} \left[ Y^{1} - Y^{0} \right] = $$

$$ = \beta $$

A intuição é que, se $\hat{p}$ está especificado corretamente, o $\hat{\beta}^{DR}$ é imparcial e o fator de ajuste desaparece, uma vez que os resíduos ($D_{i}-\hat{p}(X)$) convergem para zero.

### Aplicação em Python

In [ ]:
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np

In [ ]:
# Regressão Logística para estimar o escore de propensão
logit_model = smf.logit("Treated ~ 1 + casada + mage + medu", data=df).fit()
# Imprimindo o modelo
print(logit_model.summary())

In [ ]:
# Salvando o escore de propensão no DataFrame
df['ps'] = logit_model.predict()

In [ ]:
# Verificar graficamente a área de sobreposição
sns.histplot(data=df, x='ps', hue='Treated', bins=100, stat='density', common_norm=False).\
    set(ylabel="", title="Distribution of Propensity Scores");

Agora vamos calcular os pesos IPW para o ATE e ATT.

In [ ]:
# Inverse Probability of Treatment Weight (IPTW)

# Peso para o efeito médio do tratamento (ATE)
df['W1'] = 1 / df['ps']
df.loc[df['Treated'] == 0, 'W1'] = 0
df['W2'] = 1 / (1 - df['ps'])
df.loc[df['Treated'] == 1, 'W2'] = 0

# Peso para o efeito médio do tratamento nos tratados (ATT)
df['W_ATE'] = df['W1'] + df['W2']
df['W_ATT'] = df['ps'] / (1 - df['ps'])
df.loc[df['Treated'] == 1, 'W_ATT'] = 1

A título de curiosidade, podemos estimar o efeito médio do tratamento para os tratados (ATT) e o efeito médio do tratamento (ATE) utilizando o método de Ponderação pelo Escore de Propensão (IPW). Para isso, basta rodar a regressão linear considerando como peso amostral os valores de $W_{ATE}$ e $W_{ATT}$, respectivamente.

In [ ]:
#Propensity Score Weighting - ATE
psw_ate = smf.wls("Y ~ Treated", weights=df['W_ATE'], data=df).fit()
print(psw_ate.summary())

In [ ]:
#Propensity Score Weighting - ATT
psw_att = smf.wls("Y ~ Treated", weights=df['W_ATT'], data=df).fit()
print(psw_att.summary())

Repare que podemos obter os mesmos resultados utilizando o pacote pyDRReg.

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

IPW_att = pyDRReg(df, X_vars, T_var, Y_var, method='att', estimator='IPW', n_bootstrap=50, seed=44)

print(IPW_att.summary())

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

IPW_ate = pyDRReg(df, X_vars, T_var, Y_var, method='ate', estimator='IPW', n_bootstrap=50, seed=44)

print(IPW_ate.summary())

**Estimativa Duplamente Robusta**

Agora temos todos os componentes para estimar o DR para ATE e o ATT, vamos fazer "na mão".

In [ ]:
# DR-ATE
DR_ATE = mu1 - mu0 + df["Treated"] / df["ps"] * (df["Y"] - mu1) - (1-df["Treated"]) / (1-df["ps"]) * (df["Y"] - mu0)
print(np.mean(DR_ATE))

In [ ]:
# DR-ATT
DR_ATT = mu1 - mu0 + df["Treated"] * (df["Y"] - mu1) - (1-df["Treated"])*df["ps"] / (1-df["ps"]) * (df["Y"] - mu0)
print(np.mean(DR_ATT))

Podemos utilizar o pacote pyDRReg para obter os mesmos resultados.

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

DR_ate = pyDRReg(df, X_vars, T_var, Y_var, method='ate', estimator='DR', n_bootstrap=50, seed=44)

print(DR_ate.summary())

In [ ]:
from pyDRReg.pyDRReg import pyDRReg

T_var = 'Treated'
Y_var = 'Y'
X_vars = ['casada', 'mage', 'medu']

DR_att = pyDRReg(df, X_vars, T_var, Y_var, method='att', estimator='DR', n_bootstrap=50, seed=44)

print(DR_att.summary())

Esse tipo de estimador é bastante importante na literatura. E já possui alguns estimadores que realizam as estimações de forma direta. Por exemplo, poderíamos computar diretamente com 'LinearDRLearner' da biblioteca 'EconML' da Microsoft (EconML - Estimate causal effects with ML).

obs: https://www.microsoft.com/en-us/research/project/econml/

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from econml.dr import LinearDRLearner

In [ ]:
X = df[['casada', 'mage', 'medu']]

In [ ]:
model = LinearDRLearner(model_propensity=LogisticRegression(), 
                        model_regression=LinearRegression(),
                        random_state=1)
model.fit(Y=df["Y"], T=df["Treated"], X=X);

In [ ]:
model.ate_inference(X=X.values, T0=0, T1=1).summary().tables[0]

O modelo nos dá diretamente o efeito médio do tratamento. A estimativa é estatisticamente diferente de zero e o intervalo de confiança inclui o valor verdadeiro de -229,17. Observe que obtivemos uma estimativa diferente porque a função **LinearDRLearner** também realizou o cross-fitting em segundo plano, o que não fizemos antes. Ele não calcula o ATT.

Outro pacote importante é o "causalml" (https://causalml.readthedocs.io/en/latest/about.html).



## Boas práticas

* Verifique o balanço das covariáveis.
  * Tanto o IPW quanto o DR (AIPW) foram desenvolvidos para ambientes nos quais o tratamento não é atribuído aleatoriamente incondicionalmente, mas pode depender de algumas variáveis observáveis. Essas informações podem ser verificadas de duas maneiras: 
    * (1) Produza uma tabela de médias/equilíbrio das covariáveis. Se a randomização incondicional não for válida, esperamos ver diferenças significativas entre alguns observáveis; 
    * (2) Trace os escores de propensão estimados. Se a randomização incondicional for válida, esperamos que os escores de propensão sejam constantes.
* Verifique a suposição de sobreposição.
  * Podemos simplesmente verificar os limites dos escores de propensão previstos. Se a suposição de sobreposição for violada, acabamos dividindo algum termo do estimador por zero.